In [3]:
!pip install paddlex

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 952kB 10.1MB/s eta 0:00:01
     |████████████████████████████████| 296kB 61.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 68.0MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 38.6MB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 31.0MB/s eta 0:00:01
     |████████████████████████████████| 286kB 48.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 28.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 26.9MB/s ta 0:00:01
     |████████████████████████████████| 102kB 27.5MB/s ta 0:00:01
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1593873 sha256=3d2f4a843c22e7308f8084d01a2b3f435e73119bf8b7ae47f29b11bb8b6c1a27
  Stored in directory: /home/aistudio/.cache/pip/wheels/66/b9/1a/5c513d0b33edd38e4b95052909201336e6526c65226044faff
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-non

In [4]:
import paddlex as pdx

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[11-02 19:56:54 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of

In [22]:
from paddlex import transforms as T

train_transforms = T.Compose([
    T.MixupImage(mixup_epoch=-1),
   T.RandomDistort(),
    T.RandomExpand(im_padding_value=[123.675, 116.28, 103.53]), T.RandomCrop(),
    T.RandomHorizontalFlip(), T.BatchRandomResize(
        target_sizes=[
             800, 832, 864, 896, 928
        ],
        interp='RANDOM'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eval_transforms = T.Compose([
    T.Resize(
        target_size=864, interp='CUBIC'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [23]:
train_dataset = pdx.datasets.CocoDetection(
    data_dir='work/NFL/images',
    ann_file='work/NFL/ann_file_train.json',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='work/NFL/images',
    ann_file='work/NFL/ann_file_val.json',
    transforms=eval_transforms)

loading annotations into memory...
Done (t=0.41s)
creating index...
index created!
2021-11-02 20:16:16 [INFO]	Starting to read file list from dataset...
2021-11-02 20:16:17 [INFO]	7957 samples in file work/NFL/ann_file_train.json, including 7957 positive samples and 0 negative samples.
loading annotations into memory...
Done (t=0.11s)
creating index...
index created!
2021-11-02 20:16:17 [INFO]	Starting to read file list from dataset...
2021-11-02 20:16:17 [INFO]	1990 samples in file work/NFL/ann_file_val.json, including 1990 positive samples and 0 negative samples.


In [8]:
import copy
import os
import numpy as np
from tqdm import tqdm
from scipy.cluster.vq import kmeans
from paddlex.utils import logging

__all__ = ['YOLOAnchorCluster']


class BaseAnchorCluster(object):
    def __init__(self, num_anchors, cache, cache_path):
        """
        Base Anchor Cluster
        Args:
            num_anchors (int): number of clusters
            cache (bool): whether using cache
            cache_path (str): cache directory path
        """
        super(BaseAnchorCluster, self).__init__()
        self.num_anchors = num_anchors
        self.cache_path = cache_path
        self.cache = cache

    def print_result(self, centers):
        raise NotImplementedError('%s.print_result is not available' %
                                  self.__class__.__name__)

    def get_whs(self):
        whs_cache_path = os.path.join(self.cache_path, 'whs.npy')
        shapes_cache_path = os.path.join(self.cache_path, 'shapes.npy')
        if self.cache and os.path.exists(whs_cache_path) and os.path.exists(
                shapes_cache_path):
            self.whs = np.load(whs_cache_path)
            self.shapes = np.load(shapes_cache_path)
            return self.whs, self.shapes
        whs = np.zeros((0, 2))
        shapes = np.zeros((0, 2))
        samples = copy.deepcopy(self.dataset.file_list)
        for sample in tqdm(samples):
            im_h, im_w = sample['image_shape']
            bbox = sample['gt_bbox']
            wh = bbox[:, 2:4] - bbox[:, 0:2]
            wh = wh / np.array([[im_w, im_h]])
            shape = np.ones_like(wh) * np.array([[im_w, im_h]])
            whs = np.vstack((whs, wh))
            shapes = np.vstack((shapes, shape))

        if self.cache:
            os.makedirs(self.cache_path, exist_ok=True)
            np.save(whs_cache_path, whs)
            np.save(shapes_cache_path, shapes)

        self.whs = whs
        self.shapes = shapes
        return self.whs, self.shapes

    def calc_anchors(self):
        raise NotImplementedError('%s.calc_anchors is not available' %
                                  self.__class__.__name__)

    def __call__(self):
        self.get_whs()
        centers = self.calc_anchors()
        return centers


class YOLOAnchorCluster(BaseAnchorCluster):
    def __init__(self,
                 num_anchors,
                 dataset,
                 image_size,
                 cache=True,
                 cache_path=None,
                 iters=300,
                 gen_iters=1000,
                 thresh=0.25):
        """
        YOLOv5 Anchor Cluster
        Reference:
            https://github.com/ultralytics/yolov5/blob/master/utils/autoanchor.py
        Args:
            num_anchors (int): number of clusters
            dataset (DataSet): DataSet instance, VOC or COCO
            image_size (list or int): [h, w], being an int means image height and image width are the same.
            cache (bool): whether using cache。 Defaults to True.
            cache_path (str or None, optional): cache directory path. If None, use `data_dir` of dataset. Defaults to None.
            iters (int, optional): iters of kmeans algorithm. Defaults to 300.
            gen_iters (int, optional): iters of genetic algorithm. Defaults to 1000.
            thresh (float, optional): anchor scale threshold. Defaults to 0.25.
        """
        self.dataset = dataset
        if cache_path is None:
            cache_path = 'work/NFL/images'
        if isinstance(image_size, int):
            image_size = [image_size] * 2
        self.image_size = image_size
        self.iters = iters
        self.gen_iters = gen_iters
        self.thresh = thresh
        super(YOLOAnchorCluster, self).__init__(num_anchors, cache, cache_path)

    def print_result(self, centers):
        whs = self.whs
        x, best = self.metric(whs, centers)
        bpr, aat = (best > self.thresh).mean(), (
            x > self.thresh).mean() * self.num_anchors
        logging.info(
            'thresh=%.2f: %.4f best possible recall, %.2f anchors past thr' %
            (self.thresh, bpr, aat))
        logging.info(
            'n=%g, img_size=%s, metric_all=%.3f/%.3f-mean/best, past_thresh=%.3f-mean: '
            % (self.num_anchors, self.image_size, x.mean(), best.mean(),
               x[x > self.thresh].mean()))
        logging.info('%d anchor cluster result: [w, h]' % self.num_anchors)
        for w, h in centers:
            logging.info('[%d, %d]' % (w, h))

    def metric(self, whs, centers):
        r = whs[:, None] / centers[None]
        x = np.minimum(r, 1. / r).min(2)
        return x, x.max(1)

    def fitness(self, whs, centers):
        _, best = self.metric(whs, centers)
        return (best * (best > self.thresh)).mean()

    def calc_anchors(self):
        self.whs = self.whs * self.shapes / self.shapes.max(
            1, keepdims=True) * np.array([self.image_size[::-1]])
        wh0 = self.whs
        i = (wh0 < 3.0).any(1).sum()
        if i:
            logging.warning('Extremely small objects found. %d of %d '
                            'labels are < 3 pixels in width or height' %
                            (i, len(wh0)))

        wh = wh0[(wh0 >= 2.0).any(1)]
        logging.info('Running kmeans for %g anchors on %g points...' %
                     (self.num_anchors, len(wh)))
        s = wh.std(0)
        centers, dist = kmeans(wh / s, self.num_anchors, iter=self.iters)
        centers *= s

        f, sh, mp, s = self.fitness(wh, centers), centers.shape, 0.9, 0.1
        pbar = tqdm(
            range(self.gen_iters),
            desc='Evolving anchors with Genetic Algorithm')
        for _ in pbar:
            v = np.ones(sh)
            while (v == 1).all():
                v = ((np.random.random(sh) < mp) * np.random.random() *
                     np.random.randn(*sh) * s + 1).clip(0.3, 3.0)
            new_centers = (centers.copy() * v).clip(min=2.0)
            new_f = self.fitness(wh, new_centers)
            if new_f > f:
                f, centers = new_f, new_centers.copy()
                pbar.desc = 'Evolving anchors with Genetic Algorithm: fitness = %.4f' % f

        centers = np.round(centers[np.argsort(centers.prod(1))]).astype(int)
        return centers

In [7]:
# anchors = YOLOAnchorCluster(dataset=train_dataset,num_anchors=9, image_size=864)()
#640
# anchors=[[ 4  ,4],[ 5 , 5] ,[ 6 , 7],[ 8 , 8],[10 ,10], [13 ,12],[11, 15],[15, 16],[21, 23]]
#864
anchors=[[ 5 , 5],[ 7 , 8],[ 9, 10],[12, 12],[15 ,15],[19, 17],[16, 21],[21 ,23],[30 ,32]]
anchor_masks = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]

In [4]:
print(anchors)

[[5, 5], [7, 8], [9, 10], [12, 12], [15, 15], [19, 17], [16, 21], [21, 23], [30, 32]]


In [ ]:
num_classes = len(train_dataset.labels)
print(num_classes)
model = pdx.det.PPYOLOv2(num_classes=num_classes, 
                            backbone='ResNet101_vd_dcn',
                            label_smooth=True,
                            anchors=anchors,
                       anchor_masks=anchor_masks,
                       ignore_threshold=0.5,
                       nms_score_threshold=0.0001,
                       nms_iou_threshold=0.01)
model.train(
    num_epochs=30,
    train_dataset=train_dataset,
    train_batch_size=8,
    eval_dataset=eval_dataset,
    pretrain_weights='COCO',
    use_ema=True,
    learning_rate=0.0001,
    warmup_steps=1000,
    warmup_start_lr=0.0,
    lr_decay_epochs=[10,15,20],
    save_interval_epochs=1,
    save_dir='output/PPYOLOv2')

5


W1102 00:47:12.376727  1405 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1102 00:47:12.381939  1405 device_context.cc:422] device: 0, cuDNN Version: 7.6.


2021-11-02 00:47:15 [INFO]	Loading pretrained model from output/PPYOLOv2/pretrain/ppyolov2_r101vd_dcn_365e_coco.pdparams
2021-11-02 00:47:18 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.0.weight doesn't match.(Pretrained: [258, 1024, 1, 1], Actual: [33, 1024, 1, 1])
2021-11-02 00:47:18 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.0.bias doesn't match.(Pretrained: [258], Actual: [33])
2021-11-02 00:47:18 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.1.weight doesn't match.(Pretrained: [258, 512, 1, 1], Actual: [33, 512, 1, 1])
2021-11-02 00:47:18 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.1.bias doesn't match.(Pretrained: [258], Actual: [33])
2021-11-02 00:47:18 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.2.weight doesn't match.(Pretrained: [258, 256, 1, 1], Actual: [33, 256, 1, 1])
2021-11-02 00:47:18 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.2.bias doesn't

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:641: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.float32, the right dtype will convert to paddle.int64
  format(lhs_dtype, rhs_dtype, lhs_dtype))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:239: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.int64, the right dtype will convert to paddle.float32
  format(lhs_dtype, rhs_dtype, lhs_dtype))


2021-11-02 00:47:41 [INFO]	[TRAIN] Epoch=1/30, Step=10/994, loss_xy=21.589495, loss_wh=36.236702, loss_iou=87.499802, loss_iou_aware=11.336639, loss_obj=9617.394531, loss_cls=65.047409, loss=9839.105469, lr=0.000001, time_each_step=2.21s, eta=18:27:36
2021-11-02 00:48:02 [INFO]	[TRAIN] Epoch=1/30, Step=20/994, loss_xy=19.553185, loss_wh=41.399696, loss_iou=86.313499, loss_iou_aware=10.869906, loss_obj=1242.692627, loss_cls=64.382492, loss=1465.211304, lr=0.000002, time_each_step=2.09s, eta=17:26:40
2021-11-02 00:48:15 [INFO]	[TRAIN] Epoch=1/30, Step=30/994, loss_xy=17.719292, loss_wh=40.990440, loss_iou=79.112404, loss_iou_aware=10.346204, loss_obj=365.290649, loss_cls=58.675484, loss=572.134460, lr=0.000003, time_each_step=1.38s, eta=11:30:5
2021-11-02 00:48:37 [INFO]	[TRAIN] Epoch=1/30, Step=40/994, loss_xy=23.824347, loss_wh=44.155849, loss_iou=98.482788, loss_iou_aware=11.547204, loss_obj=288.193726, loss_cls=73.299103, loss=539.503052, lr=0.000004, time_each_step=2.18s, eta=18:9:3

/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/pycocotools/cocoeval.py:378: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tp_sum = np.cumsum(tps, axis=1).astype(dtype=np.float)
/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/pycocotools/cocoeval.py:379: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fp_sum = np.cum

DONE (t=3.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.117
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.260
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.116
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.021
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.158
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.248
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.246
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.261
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
2021-11-02 01:23:48 [INF

In [13]:
import paddlex as pdx

model = pdx.load_model('output/PPYOLOv2/best_model/')
metrics, evaluate_details = model.evaluate(eval_dataset, batch_size=1, return_details=True)

2021-11-02 19:59:12 [INFO]	Model[PPYOLOv2] loaded.
2021-11-02 19:59:15 [INFO]	Start to evaluate(total_samples=1990, total_steps=1990)...
2021-11-02 20:02:20 [INFO]	Start evaluate...
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=41.29s).
Accumulating evaluation results...


/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/pycocotools/cocoeval.py:378: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tp_sum = np.cumsum(tps, axis=1).astype(dtype=np.float)
/home/aistudio/.data/webide/pip/lib/python3.7/site-packages/pycocotools/cocoeval.py:379: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fp_sum = np.cum

DONE (t=2.57s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.223
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.421
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.052
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.279
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.407
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.402
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.511
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000


In [17]:
import matplotlib.pyplot as plt

gt = evaluate_details['gt']
bbox = evaluate_details['bbox']
pdx.det.draw_pr_curve(gt=gt, pred_bbox=bbox, save_dir='./output_nfl')

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.87s).
Accumulating evaluation results...
DONE (t=0.98s).


In [18]:
result = model.predict('work/NFL/images/57502_000480_Endzone_frame0495.jpg')
pdx.det.visualize('work/NFL/images/57502_000480_Endzone_frame0495.jpg', result, save_dir='./output_nfl')

2021-11-02 20:12:35 [INFO]	The visualized result is saved at ./output_nfl/visualize_57502_000480_Endzone_frame0495.jpg


In [19]:
pdx.det.coco_error_analysis(gt=gt, pred_bbox=bbox, save_dir='./output_nfl')

creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.89s).
Accumulating evaluation results...
DONE (t=0.88s).
--------------analyzing 1-Helmet-----------------------------analyzing 2-Helmet-Blurred-----------------------------analyzing 3-Helmet-Difficult---------------

--------------analyzing 4-Helmet-Sideline-----------------------------analyzing 5-Helmet-Partial---------------


creating index...
creating index...index created!

index created!
creating index...
index created!creating index...

index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
Running per image evaluation...
Evaluate annotation type *bbox*
Running per image evaluation...
Evaluate annotation type *bbox*
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=13.54s).
Accumulating evaluation results...
DONE (t=15.77s).
Accumulating evaluation results...
DO

In [31]:
num_classes = len(train_dataset.labels)
print(num_classes)
model = pdx.det.PPYOLOTiny(num_classes=num_classes, 
                            backbone='MobileNetV3',
                            label_smooth=True,
                            anchors=anchors,
                       anchor_masks=anchor_masks,
                       ignore_threshold=0.5,
                       nms_score_threshold=0.0001,
                       nms_iou_threshold=0.01)
model.train(
    num_epochs=20,
    train_dataset=train_dataset,
    train_batch_size=20,
    eval_dataset=eval_dataset,
    pretrain_weights='COCO',
    use_ema=True,
    learning_rate=0.005,
    warmup_steps=1000,
    warmup_start_lr=0.0,
    lr_decay_epochs=[10,15,20,25],
    save_interval_epochs=1,
    save_dir='output/PPYOLOTiny')

5
2021-11-02 22:42:59 [INFO]	Loading pretrained model from output/PPYOLOTiny/pretrain/ppyolo_tiny_650e_coco.pdparams
2021-11-02 22:42:59 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.0.weight doesn't match.(Pretrained: [255, 160, 1, 1], Actual: [30, 160, 1, 1])
2021-11-02 22:42:59 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.0.bias doesn't match.(Pretrained: [255], Actual: [30])
2021-11-02 22:42:59 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.1.weight doesn't match.(Pretrained: [255, 128, 1, 1], Actual: [30, 128, 1, 1])
2021-11-02 22:42:59 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.1.bias doesn't match.(Pretrained: [255], Actual: [30])
2021-11-02 22:42:59 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.2.weight doesn't match.(Pretrained: [255, 96, 1, 1], Actual: [30, 96, 1, 1])
2021-11-02 22:42:59 [WARNING]	[SKIP] Shape of pretrained params yolo_head.yolo_output.2.bias doesn't match.(

2021-11-02 22:46:08 ERROR: DataLoader reader thread raised an exception!
Exception in thread Thread-56:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 462, in _get_data
    data = self._data_queue.get(timeout=self._timeout)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/queues.py", line 105, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 391, in _thread_loop
    batch = self.

KeyboardInterrupt: 

In [33]:
model = pdx.load_model('output/PPYOLOv2/best_model')

2021-11-02 22:48:07 [INFO]	Model[PPYOLOv2] loaded.


In [ ]:
model.analyze_sensitivity(
    dataset=eval_dataset,
    batch_size=1,
    save_dir='output/PPYOLOv2/prune')

2021-11-02 22:48:52,149-WARNING: Leaves ['t_1191', 't_1174', 't_1154', 't_1148', 't_1126', 't_1122', 't_1102', 't_1085', 't_1084', 't_1086', 't_1075', 't_1069', 't_1061', 't_1060', 't_1059', 't_1051', 't_1045', 't_1037', 't_1180', 't_1036', 't_1035', 't_1029', 't_1028', 't_1027', 't_1020', 't_1010', 't_1003', 't_1002', 't_995', 't_994', 't_993', 't_987', 't_979', 't_977', 't_971', 't_970', 't_969', 't_963', 't_961', 't_955', 't_954', 't_953', 't_28', 't_695', 't_15', 't_7', 't_1', 'fetch_0', 't_392', 't_172', 't_203', 't_425', 't_183', 't_739', 't_594', 't_65', 't_121', 't_77', 't_96', 't_806', 't_821', 't_412', 't_417', 't_782', 't_895', 't_888', 't_1077', 't_928', 't_76', 't_21', 't_84', 't_178', 't_654', 't_737', 't_89', 't_239', 't_2', 't_632', 't_159', 't_444', 't_235', 't_457', 't_1052', 't_889', 't_650', 't_501', 't_639', 't_343', 't_1004', 't_165', 't_44', 't_450', 't_26', 't_838', 't_754', 't_486', 't_577', 't_387', 't_22', 't_672', 't_805', 't_3', 't_790', 't_273', 't_221', '

2021-11-02 22:51:10 [INFO]	Sensitivity analysis of model parameters starts...
2021-11-02 22:51:16 [INFO]	Start to evaluate(total_samples=1990, total_steps=1990)...
2021-11-02 22:54:21 [INFO]	Start evaluate...
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=41.33s).
Accumulating evaluation results...
DONE (t=2.60s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.223
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.421
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.052
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=

2021-11-02 22:55:09,874-INFO: Pruning variable [conv2d_0.w_0] and its relatives ['conv2d_0.w_0', 'batch_norm_0.w_0', 'batch_norm_0.b_0', 'batch_norm_0.w_1', 'batch_norm_0.w_2', 'conv2d_1.w_0']


2021-11-02 22:55:11 [INFO]	Start to evaluate(total_samples=1990, total_steps=1990)...
2021-11-02 22:58:18 [INFO]	Start evaluate...
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=41.55s).
Accumulating evaluation results...
DONE (t=2.63s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.223
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.421
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.212
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.314
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.052
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.279
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

2021-11-02 22:59:05,805-INFO: pruned param: conv2d_0.w_0; 0.1; loss=0.0
2021-11-02 22:59:05,808-INFO: Save status into output/PPYOLOYOLOv2/prune/model.sensi.data
2021-11-02 22:59:05,815-INFO: Pruning variable [conv2d_0.w_0] and its relatives ['conv2d_0.w_0', 'batch_norm_0.w_0', 'batch_norm_0.b_0', 'batch_norm_0.w_1', 'batch_norm_0.w_2', 'conv2d_1.w_0']


...

2021-11-03 09:10:07 [INFO]	Start to evaluate(total_samples=1990, total_steps=1990)...
2021-11-03 09:13:11 [INFO]	Start evaluate...
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=41.00s).
Accumulating evaluation results...
DONE (t=2.65s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.415
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.206
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.052
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.275
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

2021-11-03 09:13:58,292-INFO: pruned param: conv2d_24.w_0; 0.7; loss=0.019262460349430556
2021-11-03 09:13:58,295-INFO: Save status into output/PPYOLOYOLOv2/prune/model.sensi.data
2021-11-03 09:13:58,304-INFO: Pruning variable [conv2d_24.w_0] and its relatives ['conv2d_24.w_0', 'batch_norm_24.w_0', 'batch_norm_24.b_0', 'batch_norm_24.w_1', 'batch_norm_24.w_2', 'conv2d_25.w_0']


2021-11-03 09:14:01 [INFO]	Start to evaluate(total_samples=1990, total_steps=1990)...
2021-11-03 09:17:06 [INFO]	Start evaluate...
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=43.62s).
Accumulating evaluation results...
DONE (t=2.73s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.219
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.417
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.206
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.215
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.307
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.051
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.277
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

2021-11-03 09:17:54,236-INFO: pruned param: conv2d_24.w_0; 0.8; loss=0.020449832393581478
2021-11-03 09:17:54,240-INFO: Save status into output/PPYOLOYOLOv2/prune/model.sensi.data
2021-11-03 09:17:54,250-INFO: Pruning variable [conv2d_24.w_0] and its relatives ['conv2d_24.w_0', 'batch_norm_24.w_0', 'batch_norm_24.b_0', 'batch_norm_24.w_1', 'batch_norm_24.w_2', 'conv2d_25.w_0']


2021-11-03 09:17:56 [INFO]	Start to evaluate(total_samples=1990, total_steps=1990)...
2021-11-03 09:21:02 [INFO]	Start evaluate...
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


In [ ]:
# Step 2/3: according to flops prune
# API：https://github.com/PaddlePaddle/PaddleX/blob/95c53dec89ab0f3769330fa445c6d9213986ca5f/paddlex/cv/models/base.py#L394
model.prune(pruned_flops=.2)

In [ ]:
model.train(
    num_epochs=20,
    train_dataset=train_dataset,
    train_batch_size=20,
    eval_dataset=eval_dataset,
    pretrain_weights='COCO',
    use_ema=True,
    learning_rate=0.0001
    warmup_steps=1000,
    warmup_start_lr=0.0,
    lr_decay_epochs=[10,15],
    save_interval_epochs=1,
    save_dir='output/PPYOLOYOLOv2/prune')